In [15]:
# =======================================================================
# SCRIPT MAESTRO DE PREPARACIÓN DE DATOS (VERSIÓN ENRIQUECIDA)
# Usar como plantilla para cada año: 2018, 2020, 2022, 2024
# =======================================================================

import pandas as pd
import numpy as np

# --- 1. CONFIGURACIÓN INICIAL (EJEMPLO PARA 2024) ---
AÑO = '20' # Cambia esto por '18', '20', '22'
print(f"--- INICIANDO PREPARACIÓN DE DATOS 20{AÑO} ---")

ruta_base = f'../data/enigh_20{AÑO}/'
path_hogares = f'{ruta_base}hogares.csv'
path_pobreza = f'{ruta_base}pobreza{AÑO}.csv'
path_gastos = f'{ruta_base}gastoshogar.csv'
path_poblacion = f'{ruta_base}poblacion.csv' # Nueva base de datos

# --- 2. CARGA DE DATOS FUENTE ---
print("Cargando archivos...")
df_hogares = pd.read_csv(path_hogares, low_memory=False)
df_pobreza = pd.read_csv(path_pobreza, low_memory=False)
df_gastos = pd.read_csv(path_gastos, low_memory=False)
df_poblacion = pd.read_csv(path_poblacion, low_memory=False)
print("Archivos cargados.")

# --- 3. PREPARACIÓN DE BASES AGREGADAS A NIVEL HOGAR ---
print("Procesando datos...")

# A. Base de Pobreza (ahora con más variables)
columnas_pobreza = [
    'folioviv', 'pobreza', 'pobreza_e', 'ict', 'ictpc', # Ingresos
    'rururb', "factor", # Ámbito
    'ic_rezedu', 'ic_asalud', 'ic_segsoc', 'ic_cv', 'ic_sbv', 'ic_ali' # Carencias
]
df_pobreza_hogar = df_pobreza[columnas_pobreza].drop_duplicates(subset='folioviv').copy()

# B. Base de Gastos (lógica sin cambios)
clave_gasto = '083201' if AÑO == '24' else 'F003' # Lógica para clave de gasto
df_gastos['clave'] = df_gastos['clave'].astype(str)
gastos_celular = df_gastos[(df_gastos['clave'] == clave_gasto) & (df_gastos['tipo_gasto'] == 'G1')].copy()
gastos_celular['gasto_tri'] = pd.to_numeric(gastos_celular['gasto_tri'], errors='coerce')
df_gasto_agregado = gastos_celular.groupby('folioviv')['gasto_tri'].sum().reset_index()
df_gasto_agregado['gasto_celular'] = df_gasto_agregado['gasto_tri'] / 3
df_gasto_hogar = df_gasto_agregado[['folioviv', 'gasto_celular']]

# C. NUEVO: Base de Jefatura de Hogar Femenina
# El parentesco 101 usualmente identifica al jefe(a) de hogar. Verifica el diccionario si cambia.
jefes_de_hogar = df_poblacion[df_poblacion['parentesco'] == 101].copy()
# Sexo 2 usualmente es Mujer.
jefes_de_hogar['Jefatura_Mujer'] = np.where(jefes_de_hogar['sexo'] == 2, 1, 0)
df_jefatura_hogar = jefes_de_hogar[['folioviv', 'Jefatura_Mujer']]

print("Datos procesados a nivel hogar.")

# --- 4. UNIÓN DE TODAS LAS BASES DE DATOS ---
print("Uniendo las bases de datos...")
# Empezamos con la base de hogares
df_final = df_hogares
# Unimos pobreza y las nuevas variables
df_final = pd.merge(df_final, df_pobreza_hogar, on='folioviv', how='left')
# Unimos gasto
df_final = pd.merge(df_final, df_gasto_hogar, on='folioviv', how='left')
# Unimos la nueva variable de jefatura
df_final = pd.merge(df_final, df_jefatura_hogar, on='folioviv', how='left')

# Limpieza final
df_final['gasto_celular'].fillna(0, inplace=True)
df_final['Jefatura_Mujer'].fillna(0, inplace=True) # Asumimos 0 si no se encuentra jefa

print(f"Unión de datos para 20{AÑO} finalizada.")

# --- 5. GUARDAR EL DATAFRAME ENRIQUECIDO ---
ruta_guardado = '../data/procesados/'
if not os.path.exists(ruta_guardado):
    os.makedirs(ruta_guardado)

archivo_parquet = f'{ruta_guardado}enigh_20{AÑO}_final_enriquecido.parquet'
df_final.to_parquet(archivo_parquet)

print(f"¡Trabajo de 20{AÑO} completado y guardado en: {archivo_parquet}!")

--- INICIANDO PREPARACIÓN DE DATOS 2020 ---
Cargando archivos...
Archivos cargados.
Procesando datos...
Datos procesados a nivel hogar.
Uniendo las bases de datos...
Unión de datos para 2020 finalizada.
¡Trabajo de 2020 completado y guardado en: ../data/procesados/enigh_2020_final_enriquecido.parquet!


In [16]:
# --- CÓDIGO DE VERIFICACIÓN DE ARCHIVOS ENRIQUECIDOS ---
import pandas as pd

años = [2018, 2020, 2022, 2024]
ruta_base = '../data/procesados/'

print("--- Verificando la presencia de la columna 'factor' en los archivos procesados ---")

for año in años:
    # Construir el nombre del archivo
    nombre_archivo = f'enigh_{año}_final_enriquecido.parquet'
    ruta_completa = f'{ruta_base}{nombre_archivo}'
    
    try:
        # Cargar el archivo
        df_temp = pd.read_parquet(ruta_completa)
        
        # Verificar si la columna 'factor' existe
        if 'factor' in df_temp.columns:
            print(f"✅ Archivo de {año}: La columna 'factor' SÍ está presente.")
        else:
            print(f"❌ Archivo de {año}: ¡ALERTA! La columna 'factor' NO está presente.")
            
    except FileNotFoundError:
        print(f"⚠️  Archivo de {año}: No se encontró el archivo '{nombre_archivo}'.")

--- Verificando la presencia de la columna 'factor' en los archivos procesados ---
✅ Archivo de 2018: La columna 'factor' SÍ está presente.
✅ Archivo de 2020: La columna 'factor' SÍ está presente.
✅ Archivo de 2022: La columna 'factor' SÍ está presente.
✅ Archivo de 2024: La columna 'factor' SÍ está presente.
